<h1>Index<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import pdb
import requests
import time
import random
import os
import glob
import sys
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select
from tqdm import tqdm_notebook, tqdm
from joblib import Parallel, delayed

In [2]:
SAVING_PATH = "~/Documents/datasets/cartola"
RODADA = 38

In [3]:
TEAM_ABBREV_TO_TEAM_KEY = {"CAP": "athletico-pr",
                           "ACG": "atletico-go",
                           "ATL": "atletico-mg",
                           "BAH": "bahia",
                           "BGT": "bragantino",
                           "BOT": "botafogo",
                           "CEA": "ceara",
                           "COR": "corinthians",
                           "CFC": "coritiba",
                           "FLA": "flamengo",
                           "FLU": "fluminense",
                           "FOR": "fortaleza",
                           "GOI": "goias",
                           "GRE": "gremio",
                           "INT": "internacional",
                           "PAL": "palmeiras",
                           "SAN": "santos",
                           "SAO": "sao-paulo",
                           "SPO": "sport",
                           "VAS": "vasco"}

TEAM_LONG_TO_TEAM_KEY = {"Athlético-PR": "athletico-pr",
                         "Atlético-GO": "atletico-go",
                         "Atlético-MG": "atletico-mg",
                         "Bahia": "bahia",
                         "Bragantino": "bragantino",
                         "Botafogo": "botafogo",
                         "Ceará": "ceara",
                         "Corinthians": "corinthians",
                         "Coritiba": "coritiba",
                         "Flamengo": "flamengo",
                         "Fluminense": "fluminense",
                         "Fortaleza": "fortaleza",
                         "Goiás": "goias",
                         "Grêmio": "gremio",
                         "Internacional": "internacional",
                         "Palmeiras": "palmeiras",
                         "Santos": "santos",
                         "São Paulo": "sao-paulo",
                         "Sport": "sport",
                         "Vasco": "vasco"}



In [4]:
def go_to_page_number(driver, page_number):
    try:
        driver.find_element_by_link_text(str(page_number)).click()
        time.sleep(10)
        return driver
    except:
        return driver
    

def get_page_numbers(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')
    tr = soup.find("tr", {"class": "tbpaging"})
    tr = tr.find("tr")
    tds = tr.find_all("td")
    page_numbers = [td.text for td in tds]
    
    return page_numbers

    
def get_table_from_page(driver):
    soup = BeautifulSoup(driver.page_source, 'lxml')

#    table = soup.findAll("table", {"class": "gridView"})
    table = soup.find("table", {"id": "ctl00_cphMainContent_gvList"})
    df = pd.read_html(str(table))[0]
    df_n_rows = df.shape[0]
    df = df.drop([df_n_rows-2, df_n_rows-1])
    imgs = table.find_all("img")
    clubs = [img.__dict__["attrs"]["alt"] for img in imgs]
    df["Clube"] = clubs
    return df

def input_element_id_with(driver, element_id, value):
    dropdown = Select(driver.find_element_by_id(element_id)) 
    dropdown.select_by_visible_text(value)
    
    return driver
    
def input_search_and_click(search, driver):
    for key, value in search.items():
        input_element_id_with(driver, key, value) 
    
    ### Click the filter button    
    driver.find_element_by_id("ctl00_cphMainContent_btnFiltrar").click()
    time.sleep(10)
    
    return driver

def get_all_tables_from_driver(driver):
    #drive = input_search_and_click(search, driver)
    
    page_numbers = get_page_numbers(driver)
            
    infos = []
    
    for page_number in page_numbers:
        go_to_page_number(driver, page_number)
        df = get_table_from_page(driver)
        infos.append(df)

        
    return pd.concat(infos)


def go_to_specific_tab(driver, tab_name):
    try:
        driver.find_element_by_link_text(tab_name).click()
        time.sleep(10)
        return driver
    except:
        return driver

def get_all_div_id_table_rows(driver, div_id):
    soup = BeautifulSoup(driver.page_source, 'lxml')    
    table = soup.find("div", {"id": div_id})
        
    return table.find_all("a", recursive=True)

def from_game_table_row_to_dict(n_row, row):
    row_data = {}
    row_data["mandante"] = row.__dict__["attrs"]["title"].split(" x ")[0]
    row_data["visitante"] = row.__dict__["attrs"]["title"].split(" x ")[1].split(" - ")[0]
    row_data["placar_mandante"] = row.find("span", {"id": "ctl00_cphMainContent_rptListPartidas_ctl{}_lblPlacarMandante".format(str(n_row).zfill(2))}).text
    row_data["placar_visitante"] = row.find("span", {"id": "ctl00_cphMainContent_rptListPartidas_ctl{}_lblPlacarVisitante".format(str(n_row).zfill(2))}).text
    row_data["data"] = row.find_all("strong")[0].text.replace("\n", "").strip()
    row_data["local"] = row.find_all("span", {"class": "text-muted"})[0].text.replace("\n", "").strip()
    
    return row_data

def extract_games_result(driver):
    go_to_specific_tab(driver, "Partidas")
    rows = get_all_div_id_table_rows(driver, "tab2default")
    games = []
    for n_row, row in enumerate(rows):    
        row_data = from_game_table_row_to_dict(n_row, row)
        games.append(row_data)

    games = pd.DataFrame.from_dict(games)
    return games

def from_player_table_row_to_dict(n_row, row):
    row_data = {}
    row_data["colocação"] = n_row + 1
    row_data["jogador"] = row.find("span", {"id": "ctl00_cphMainContent_rptListMaisEscalados_ctl{}_lblNome".format(str(n_row).zfill(2))}).text
    row_data["escalações"] = row.find("span", {"id": "ctl00_cphMainContent_rptListMaisEscalados_ctl{}_lblQtdEscalacoes".format(str(n_row).zfill(2))}).text.replace("escalações", "").replace(".", "")

    
    return row_data

def extract_most_chosen_players(driver):
    go_to_specific_tab(driver, "Mais escalados")
    rows = get_all_div_id_table_rows(driver, "tab3default") 
    players = []
    for n_row, row in enumerate(rows):    
        row_data = from_player_table_row_to_dict(n_row, row)
        players.append(row_data)
    
    players = pd.DataFrame.from_dict(players)
    #pdb.set_trace()
    try:
        players["escalações"] = players["escalações"].astype(int)
    except:
        pass
    return players
    

def get_data_from_championship_round(year, championship_round, search, driver=None, close_driver=False):
    page_prefix = "https://www.cartolafcbrasil.com.br/scouts/cartola-fc-{}/rodada-{}"
    page = page_prefix.format(year, championship_round)
    
    if not driver:
        options = Options()
        options.add_argument("--headless")
        driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
        driver.implicitly_wait(20)
        
    driver.get(page)    
    driver = input_search_and_click(search, driver)
    data = get_all_tables_from_driver(driver)
    
    if close_driver: driver.close()
        
    return data

def get_all_year_data(year, max_round, search):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Firefox(options=options, executable_path="/usr/local/bin/geckodriver")
    driver.implicitly_wait(20)
    
    all_players = []
    all_games = []
    all_uses = []
    for ch_round in range(1, max_round+1)[::-1]:
        print(ch_round)
        players = get_data_from_championship_round(year,
                                         ch_round,
                                         search,
                                         driver=driver)       
        games = extract_games_result(driver)
        uses = extract_most_chosen_players(driver)        
        players["rodada"] = ch_round
        games["rodada"] = ch_round
        uses["rodada"] = ch_round
        
        all_players.append(players)
        all_games.append(games)
        all_uses.append(uses)
        
    all_players = pd.concat(all_players[::-1])
    all_games = pd.concat(all_games[::-1])
    all_uses = pd.concat(all_uses[::-1])
    
    all_players["ano"] = year
    all_games["ano"] = year
    all_uses["ano"] = year
    
    driver.close()
    
    return all_players, all_games, all_uses

In [5]:
search = {"ctl00_cphMainContent_drpStatus": "Provável"}
#search = {"ctl00_cphMainContent_drpStatus": "[TODOS]"}

In [6]:
players_ = pd.read_csv("jogadores_2019.csv", index_col=False)
games_ = pd.read_csv("jogos_2019.csv", index_col=False)
uses_ = pd.read_csv("escalacoes_2019.csv", index_col=False)

In [7]:
players_.head()

,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,RB,G,A,...,GC,CV,CA,PP,GS,FC,I,PE,rodada,ano
0,Ricardo Bueno (ATA),Ceará,1852,1,2270.0,2270.0,1452.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,2.0,1.0,1.0,1,2019
1,Bruno Henrique (ATA),Flamengo,2151,1,2100.0,2100.0,1151.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1,2019
2,Nikão (MEI),Athlético-PR,1929,1,1880.0,1880.0,1029.0,5.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1,2019
3,Tiepo (GOL),Chapecoense,1318,1,1700.0,1700.0,1118.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2019
4,Vanderlei (GOL),Santos,2269,1,1570.0,1570.0,569.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1,2019


In [8]:
players, games, uses = get_all_year_data(2020, RODADA, search)

38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


In [9]:
final_players = pd.concat([players, players_])
final_games = pd.concat([games, games_])
final_uses = pd.concat([uses, uses_])

In [10]:
final_players.head()

,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,DS,G,A,...,CA,PP,GS,FC,I,PI,rodada,ano,RB,PE
0,Elton (ATA),Sport,1421,1.0,1560.0,1560.0,1021.0,NaN,2.0,NaN,...,1.0,NaN,NaN,1.0,1.0,4.0,1,2020,NaN,NaN
1,Saravia (LAT),Internacional,1319,1.0,1220.0,1220.0,719.0,8.0,NaN,NaN,...,1.0,NaN,NaN,3.0,NaN,8.0,1,2020,NaN,NaN
2,Marinho (ATA),Santos,1210,1.0,1170.0,1170.0,710.0,1.0,1.0,NaN,...,NaN,NaN,NaN,2.0,NaN,5.0,1,2020,NaN,NaN
3,Igor Rabello (ZAG),Atlético-MG,1341,1.0,1050.0,1050.0,541.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,1,2020,NaN,NaN
4,Léo Cittadini (MEI),Athlético-PR,1290,1.0,980.0,980.0,490.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2020,NaN,NaN


In [11]:
final_players[["Preço", "Média", "Últ. Pont.", "Variação"]] = final_players[["Preço", "Média", "Últ. Pont.", "Variação"]] / 100.0

In [12]:
final_players.head()

,Nome,Clube,Preço,J,Média,Últ. Pont.,Variação,DS,G,A,...,CA,PP,GS,FC,I,PI,rodada,ano,RB,PE
0,Elton (ATA),Sport,14.21,1.0,15.6,15.6,10.21,NaN,2.0,NaN,...,1.0,NaN,NaN,1.0,1.0,4.0,1,2020,NaN,NaN
1,Saravia (LAT),Internacional,13.19,1.0,12.2,12.2,7.19,8.0,NaN,NaN,...,1.0,NaN,NaN,3.0,NaN,8.0,1,2020,NaN,NaN
2,Marinho (ATA),Santos,12.10,1.0,11.7,11.7,7.10,1.0,1.0,NaN,...,NaN,NaN,NaN,2.0,NaN,5.0,1,2020,NaN,NaN
3,Igor Rabello (ZAG),Atlético-MG,13.41,1.0,10.5,10.5,5.41,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,1,2020,NaN,NaN
4,Léo Cittadini (MEI),Athlético-PR,12.90,1.0,9.8,9.8,4.90,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2020,NaN,NaN


In [13]:
final_players.to_csv("~/Documents/datasets/cartola/jogadores.csv", index=False)
final_games.to_csv("~/Documents/datasets/cartola/jogos.csv", index=False)
final_uses.to_csv("~/Documents/datasets/cartola/escalacoes.csv", index=False)